In [ ]:
#importing all the needed dependencies
import import_ipynb
import numpy as np #for matrix calculations
import csv
import os
from keras import layers
from keras.layers import Input, Dense,Activation, ZeroPadding2D, Flatten, Conv2D, BatchNormalization
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd #for converting csv into our dataframe
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

In [ ]:
data=pd.read_csv(r"C:\Users\Lenovo\Project 1 data.csv") #reading our csv data
dataset=np.array(data) #converting the data into an array
np.random.shuffle(dataset) #shuffling the dataset
X=dataset #x rows
Y=dataset #y columns
X=X[:, 0:1024]
Y=Y[:, 1024]

X_train=X[0:70000, :] #splitting the dataset into training and testing datasets
X_train=X_train/255 #normalising train
X_test=X[70000:72001, :]
X_test=X_test/255 #normalising test

#reshaping
Y=Y.reshape(Y.shape[0], 1)
Y_train=Y[0:70000, :]
Y_train=Y_train.T #taking transpose 
Y_test=Y[70000:72001, :]
Y_test=Y_test.T

In [ ]:
print("number of training examples=" + str(X_train.shape[0])) #printing our created dataset's information 
print("number of testing examples=" + str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))

In [ ]:
image_x=32 #image size of 32x32(height and width)
image_y=32

train_y=np_utils.to_categorical(Y_train) #converting into vectors
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0], image_x, image_y, 1)
X_test=X_test.reshape(X_test.shape[0], image_x, image_y, 1)

In [ ]:
print("X_train shape:" + str(X_train.shape)) #printing the shape of train x and train y
print("Y_train shape:" + str(train_y.shape))

In [ ]:
#building model
def keras_model(image_x, image_y): #function1
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten()) #flattening vector into a vector
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath="devanagari.h5" #saving model after building
    checkpoint1=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list=[checkpoint1]
    
    return model, callbacks_list #returning to the calling variables

In [ ]:
model, callbacks_list=keras_model(image_x, image_y) #calling the function
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=2, batch_size=64, callbacks=callbacks_list)
scores=model.evaluate(X_test, test_y, verbose=0)
print("CNN error :%.2f%%" % (100-scores[1]*100)) #printing error
print_summary(model) #printing the details of the model
model.save('devanagari.h5') #saving the model